In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
%matplotlib inline
import scipy.stats as stats
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

C:\Users\hp\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

In [3]:
data=pd.read_csv('C:\\Users\\hp\\Desktop\\analytixlabs\\Ml case study\\network intru\\NSL_Dataset\\Train.txt',header=None)

In [4]:
data.columns=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land", "wrong_fragment","urgent","hot","num_failed_logins","logged_in", "num_compromised","root_shell","su_attempted","num_root","num_file_creations", "num_shells","num_access_files","num_outbound_cmds","is_host_login", "is_guest_login","count","srv_count","serror_rate", "srv_serror_rate", "rerror_rate","srv_rerror_rate","same_srv_rate", "diff_srv_rate",
             "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate", "dst_host_diff_srv_rate","dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate", "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack", "last_flag"]

In [5]:
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,last_flag
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  int64  
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  int64  
 5   dst_bytes                    125973 non-null  int64  
 6   land                         125973 non-null  int64  
 7   wrong_fragment               125973 non-null  int64  
 8   urgent                       125973 non-null  int64  
 9   hot                          125973 non-null  int64  
 10  num_failed_logins            125973 non-null  int64  
 11  logged_in                    125973 non-null  int64  
 12  num_compromised              125973 non-null  int64  
 13 

In [7]:
def continuous_var_summary(x):
    return pd.Series([x.count(), x.isnull().sum(), x.sum(), x.mean(), x.median(),  
                      x.std(), x.var(), x.min(), x.quantile(0.01), x.quantile(0.05),
                          x.quantile(0.10),x.quantile(0.25),x.quantile(0.50),x.quantile(0.75), 
                              x.quantile(0.90),x.quantile(0.95), x.quantile(0.99),x.max()], 
                  index = ['N', 'NMISS', 'SUM', 'MEAN','MEDIAN', 'STD', 'VAR', 'MIN', 'P1', 
                               'P5' ,'P10' ,'P25' ,'P50' ,'P75' ,'P90' ,'P95' ,'P99' ,'MAX'])

In [8]:
conti_var=data.loc[:,(data.dtypes=='int64')|(data.dtypes=='float64')]

In [9]:
categ_var=data.loc[:,data.dtypes=='object']

In [10]:
conti_var.apply(lambda x:continuous_var_summary(x)).T

,N,NMISS,SUM,MEAN,MEDIAN,STD,VAR,MIN,P1,P5,P10,P25,P50,P75,P90,P95,P99,MAX
duration,125973.0,0.0,3.617247e+07,287.144650,0.00,2.604515e+03,6.783500e+06,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,4.00,9590.56,4.290800e+04
src_bytes,125973.0,0.0,5.740179e+09,45566.743000,44.00,5.870331e+06,3.446079e+13,0.0,0.0,0.00,0.00,0.00,44.00,276.00,848.00,1480.00,54540.00,1.379964e+09
dst_bytes,125973.0,0.0,2.491634e+09,19779.114421,0.00,4.021269e+06,1.617061e+13,0.0,0.0,0.00,0.00,0.00,0.00,516.00,3375.80,8314.00,25519.00,1.309937e+09
land,125973.0,0.0,2.500000e+01,0.000198,0.00,1.408607e-02,1.984174e-04,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.000000e+00
wrong_fragment,125973.0,0.0,2.858000e+03,0.022687,0.00,2.535300e-01,6.427745e-02,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.000000e+00
urgent,125973.0,0.0,1.400000e+01,0.000111,0.00,1.436603e-02,2.063827e-04,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.000000e+00
hot,125973.0,0.0,2.575000e+04,0.204409,0.00,2.149968e+00,4.622364e+00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,7.700000e+01
num_failed_logins,125973.0,0.0,1.540000e+02,0.001222,0.00,4.523914e-02,2.046580e-03,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,5.000000e+00
logged_in,125973.0,0.0,4.985200e+04,0.395736,0.00,4.890101e-01,2.391308e-01,0.0,0.0,0.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.000000e+00
num_compromised,125973.0,0.0,3.517800e+04,0.279250,0.00,2.394204e+01,5.732214e+02,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,7.479000e+03


In [11]:
conti_var=conti_var.apply(lambda x:x.clip(lower=x.quantile(0.01),upper=x.quantile(0.99)))

In [12]:
def create_dummies(df, colname):
    col_dummies = pd.get_dummies(df[colname], prefix = colname, drop_first = True)
    df = pd.concat([df, col_dummies], axis = 1)
    df.drop(colname, axis = 1, inplace = True )
    return df

In [13]:
for c_feature in ['protocol_type','service','flag']:
    categ_var[c_feature] = categ_var[c_feature].astype('category')
    categ_var = create_dummies(categ_var, c_feature)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
categ_var.head()

,attack,protocol_type_tcp,protocol_type_udp,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,service_csnet_ns,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,normal,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,normal,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,neptune,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,normal,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,normal,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
data2=pd.concat([conti_var,categ_var],axis=1)

In [16]:
data2.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,0.0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
data2['attack']=np.where((data2.attack!='normal'),1,0)

In [18]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Columns: 121 entries, duration to flag_SH
dtypes: float64(16), int32(1), int64(23), uint8(81)
memory usage: 47.7 MB


In [19]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [20]:
feature_col=data2.columns.difference(['attack'])

In [70]:
logreg=LogisticRegression(n_jobs=-1)
rfe=RFE(logreg,n_features_to_select=75,verbose=10)

In [71]:
rfe.fit(data2[feature_col],data2.attack)
print(rfe.support_)

Fitting estimator with 120 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Fitting estimator with 119 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Fitting estimator with 118 features.


...

Fitting estimator with 77 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


Fitting estimator with 76 features.


C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


[False  True False False  True  True  True  True  True False  True False
  True False  True  True  True  True  True  True  True  True  True  True
  True False False False  True False False  True False False False False
 False False False False False  True  True  True  True False  True  True
  True  True  True False False  True  True  True  True  True  True  True
  True  True False  True False  True  True False  True False  True  True
  True  True  True  True False  True  True  True False  True  True  True
  True False False False  True  True False  True False  True  True False
  True False  True False  True  True  True  True False  True False  True
  True  True False  True False False  True  True  True False False False]


In [72]:
print(rfe.support_)
print(rfe.ranking_)

[False  True False False  True  True  True  True  True False  True False
  True False  True  True  True  True  True  True  True  True  True  True
  True False False False  True False False  True False False False False
 False False False False False  True  True  True  True False  True  True
  True  True  True False False  True  True  True  True  True  True  True
  True  True False  True False  True  True False  True False  True  True
  True  True  True  True False  True  True  True False  True  True  True
  True False False False  True  True False  True False  True  True False
  True False  True False  True  True  True  True False  True False  True
  True  True False  True False False  True  True  True False False False]
[21  1 28 24  1  1  1  1  1 25  1 16  1 27  1  1  1  1  1  1  1  1  1  1
  1 39 42 37  1 10 35  1 41 34 36 43 40 18 19 17 38  1  1  1  1 30  1  1
  1  1  1  2  6  1  1  1  1  1  1  1  1  1  7  1 31  1  1 33  1 32  1  1
  1  1  1  1  8  1  1  1  4  1  1  1  1  9 20 11  

In [73]:
x_var=data2[feature_col[rfe.support_]]

In [74]:
fdata=pd.concat([x_var,data2.attack],axis=1)

In [75]:
feature='+'.join(x_var.columns)

In [76]:
a,b=dmatrices( 'attack ~'+ feature,data=fdata,return_type='dataframe')

In [77]:
vif=pd.DataFrame()
vif['vif']= [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif["features"] = b.columns
vif

,vif,features
0,2947.403680,Intercept
1,2.260686,diff_srv_rate
2,2.657869,dst_host_diff_srv_rate
3,8.550030,dst_host_rerror_rate
4,2.786338,dst_host_same_src_port_rate
...,...,...
71,1.081299,service_uucp_path
72,1.081326,service_whois
73,1.683261,srv_diff_host_rate
74,78.189828,srv_rerror_rate


In [79]:
vif[vif['vif']>5].features.values

array(['Intercept', 'dst_host_rerror_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'flag_REJ',
       'flag_RSTO', 'flag_RSTR', 'flag_S0', 'flag_S1', 'flag_SF',
       'flag_SH', 'same_srv_rate', 'serror_rate', 'service_http',
       'srv_rerror_rate', 'srv_serror_rate'], dtype=object)

In [80]:
fdata.drop(columns=[ 'dst_host_rerror_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'flag_REJ',
       'flag_RSTO', 'flag_RSTR', 'flag_S0', 'flag_S1', 'flag_SF',
       'flag_SH', 'same_srv_rate', 'serror_rate', 'service_http',
       'srv_rerror_rate', 'srv_serror_rate'],inplace=True)

In [81]:
fdata.shape

(125973, 60)

In [82]:
faeture_col2=fdata.columns.difference(['attack'])

In [83]:
train_X, test_X, train_y, test_y = train_test_split(fdata[faeture_col2],
                    fdata['attack'], test_size = 0.3, random_state = 123)

In [84]:
log_model = LogisticRegression( solver = 'liblinear',random_state=123 ).fit( train_X, train_y )

In [85]:
log_model.score(train_X, train_y )

0.9301890429911205

In [86]:
log_model.score(test_X, test_y )

0.9279212531752752

In [87]:
a1=log_model.predict_proba(train_X)[:,1]


In [88]:
b1=log_model.predict_proba(test_X)[:,1]

In [89]:
train_auc = metrics.roc_auc_score(train_y,a1)
test_auc = metrics.roc_auc_score(test_y,b1)

print("The AUC for the model built on the Train Data is : ", train_auc)
print("The AUC for the model built on the Test Data is : ", test_auc)

The AUC for the model built on the Train Data is :  0.9728032346629818
The AUC for the model built on the Test Data is :  0.9711693788537464


In [90]:
somersd_train=2*train_auc-1
somersd_test=2*test_auc-1
print("The somerd for the model built on the Train Data is : ", somersd_train)
print("The somerd for the model built on the Test Data is : ",somersd_test )


The somerd for the model built on the Train Data is :  0.9456064693259636
The somerd for the model built on the Test Data is :  0.9423387577074929


In [91]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score

In [92]:
c=log_model.predict(train_X)
d=log_model.predict(test_X)

In [93]:
confusion_matrix(train_y,c)

array([[44564,  2617],
       [ 3539, 37461]], dtype=int64)

In [94]:
confusion_matrix(test_y,d)

array([[18978,  1184],
       [ 1540, 16090]], dtype=int64)

In [95]:
precision_score(train_y,c)

0.9347023304556116

In [96]:
precision_score(test_y,d)

0.931457682065532

In [97]:
recall_score(train_y,c)

0.9136829268292683

In [98]:
recall_score(test_y,d)

0.9126488939307997

In [99]:
f1_score(train_y,c)

0.9240731147783616

In [100]:
f1_score(test_y,d)

0.9219573687829475

In [101]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [105]:
param_grid = {'max_depth':[20,25,30], 
                 'max_features':[20,25,30]}

In [106]:
tree = GridSearchCV( DecisionTreeClassifier(random_state = 123), param_grid, 
                        cv = 5, scoring = 'f1',verbose=10).fit( train_X, train_y )

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] max_depth=20, max_features=20 ...................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....... max_depth=20, max_features=20, score=0.994, total=   0.3s
[CV] max_depth=20, max_features=20 ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ....... max_depth=20, max_features=20, score=0.991, total=   0.3s
[CV] max_depth=20, max_features=20 ...................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    2.1s remaining:    0.0s


[CV] ....... max_depth=20, max_features=25, score=0.993, total=   0.4s
[CV] max_depth=20, max_features=25 ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.4s remaining:    0.0s


[CV] ....... max_depth=20, max_features=25, score=0.994, total=   0.3s
[CV] max_depth=20, max_features=30 ...................................
[CV] ....... max_depth=20, max_features=30, score=0.993, total=   0.4s
[CV] max_depth=20, max_features=30 ...................................
[CV] ....... max_depth=20, max_features=30, score=0.993, total=   0.3s
[CV] max_depth=20, max_features=30 ...................................
[CV] ....... max_depth=20, max_features=30, score=0.992, total=   0.3s
[CV] max_depth=20, max_features=30 ...................................
[CV] ....... max_depth=20, max_features=30, score=0.994, total=   0.3s
[CV] max_depth=20, max_features=30 ...................................
[CV] ....... max_depth=20, max_features=30, score=0.993, total=   0.3s
[CV] max_depth=25, max_features=20 ...................................
[CV] ....... max_depth=25, max_features=20, score=0.993, total=   0.2s
[CV] max_depth=25, max_features=20 ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   13.3s finished


In [107]:
print('Best score: ', tree.best_score_)
print('Best parameters: ', tree.best_params_)

Best score:  0.9933609630748906
Best parameters:  {'max_depth': 25, 'max_features': 25}


In [108]:
model=tree.best_estimator_.fit(train_X, train_y)

In [109]:
model.score(train_X, train_y)

0.9980267858155385

In [110]:
model.score(test_X, test_y)

0.9940992802709568

In [111]:
e=model.predict(train_X)
f=model.predict(test_X)


In [112]:
confusion_matrix(train_y,e)

array([[47064,   117],
       [   57, 40943]], dtype=int64)

In [113]:
confusion_matrix(test_y,f)

array([[20029,   133],
       [   90, 17540]], dtype=int64)

In [114]:
precision_score(train_y,e)

0.9971505114466634

In [115]:
precision_score(test_y,f)

0.9924743959712556

In [116]:
recall_score(train_y,e)

0.998609756097561

In [117]:
recall_score(test_y,f)

0.994895065229722

In [118]:
f1_score(train_y,e)

0.9978796002924689

In [119]:
f1_score(test_y,f)

0.9936832563804776

In [120]:
from sklearn.ensemble import RandomForestClassifier

In [130]:
pargrid_rf = {'n_estimators': [118,120,125,130],
                  'max_features': [25],'max_depth': [25]}

In [131]:
gscv_rf = GridSearchCV(estimator = RandomForestClassifier(random_state=123), 
                        param_grid = pargrid_rf, 
                        cv = 5,
                        verbose = 10, 
                        n_jobs = -1,scoring='roc_auc')

In [132]:
gscv_results = gscv_rf.fit(train_X, train_y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   59.4s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  16 out of  20 | elapsed:  2.1min remaining:   31.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.6min finished


In [133]:
gscv_results.best_params_

{'max_depth': 25, 'max_features': 25, 'n_estimators': 120}

In [134]:
gscv_results.best_score_

0.9997722533266071

In [135]:
model2=gscv_results.best_estimator_.fit(train_X, train_y)

In [136]:
model2.score(train_X, train_y)

0.9980834873725632

In [137]:
model2.score(test_X, test_y)


0.9957398391193903

# now importing the actual given test data and doing predictions.

In [138]:
tdata=pd.read_csv('C:\\Users\\hp\\Desktop\\analytixlabs\\Ml case study\\network intru\\NSL_Dataset\\Test.txt',header=None)

In [139]:
tdata.columns=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land", "wrong_fragment","urgent","hot","num_failed_logins","logged_in", "num_compromised","root_shell","su_attempted","num_root","num_file_creations", "num_shells","num_access_files","num_outbound_cmds","is_host_login", "is_guest_login","count","srv_count","serror_rate", "srv_serror_rate", "rerror_rate","srv_rerror_rate","same_srv_rate", "diff_srv_rate",
             "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate", "dst_host_diff_srv_rate","dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate", "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack", "last_flag"]

In [140]:
tdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22544 entries, 0 to 22543
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     22544 non-null  int64  
 1   protocol_type                22544 non-null  object 
 2   service                      22544 non-null  object 
 3   flag                         22544 non-null  object 
 4   src_bytes                    22544 non-null  int64  
 5   dst_bytes                    22544 non-null  int64  
 6   land                         22544 non-null  int64  
 7   wrong_fragment               22544 non-null  int64  
 8   urgent                       22544 non-null  int64  
 9   hot                          22544 non-null  int64  
 10  num_failed_logins            22544 non-null  int64  
 11  logged_in                    22544 non-null  int64  
 12  num_compromised              22544 non-null  int64  
 13  root_shell      

In [141]:
conti_var_test=tdata.loc[:,(tdata.dtypes=='int64')|(tdata.dtypes=='float64')].reset_index()

In [142]:
categ_var_test=tdata.loc[:,tdata.dtypes=='object'].reset_index()

In [143]:
conti_var_test=conti_var_test.apply(lambda x:x.clip(lower=x.quantile(0.01),upper=x.quantile(0.99)))

In [144]:
for c_feature in ['protocol_type','service','flag']:
    categ_var_test[c_feature] = categ_var_test[c_feature].astype('category')
    categ_var_test = create_dummies(categ_var_test, c_feature)

In [145]:
categ_var_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22544 entries, 0 to 22543
Data columns (total 77 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                22544 non-null  int64 
 1   attack               22544 non-null  object
 2   protocol_type_tcp    22544 non-null  uint8 
 3   protocol_type_udp    22544 non-null  uint8 
 4   service_X11          22544 non-null  uint8 
 5   service_Z39_50       22544 non-null  uint8 
 6   service_auth         22544 non-null  uint8 
 7   service_bgp          22544 non-null  uint8 
 8   service_courier      22544 non-null  uint8 
 9   service_csnet_ns     22544 non-null  uint8 
 10  service_ctf          22544 non-null  uint8 
 11  service_daytime      22544 non-null  uint8 
 12  service_discard      22544 non-null  uint8 
 13  service_domain       22544 non-null  uint8 
 14  service_domain_u     22544 non-null  uint8 
 15  service_echo         22544 non-null  uint8 
 16  serv

In [146]:
data3=pd.concat([conti_var_test,categ_var_test],axis=1)

In [147]:
data3.shape

(22544, 117)

In [148]:
xvar=data3[train_X.columns]

In [149]:
xvar.shape

(22544, 59)

In [150]:
yvar=np.where((data3.attack!='normal'),1,0)

In [151]:
yvar=pd.Series(yvar)

In [152]:
pred_attack=pd.Series(log_model.predict(xvar))

In [153]:
pred_attack

0        1
1        1
2        0
3        1
4        1
        ..
22539    0
22540    0
22541    1
22542    0
22543    1
Length: 22544, dtype: int32

In [154]:
metrics.accuracy_score(pred_attack,yvar)

0.8992193044712562

In [161]:
pred_attack2=pd.Series(model2.predict(xvar))

In [162]:
metrics.accuracy_score(pred_attack2,yvar)

0.7883250532292406

In [163]:
pd.concat([xvar,yvar,pred_attack],axis=1)

,diff_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,flag_RSTOS0,flag_S2,flag_S3,hot,last_flag,num_compromised,...,service_telnet,service_tftp_u,service_time,service_urp_i,service_uucp,service_uucp_path,service_whois,srv_diff_host_rate,0,1
0,0.06,0.06,0.00,0.00,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0.00,1,1
1,0.06,0.06,0.00,0.00,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0.00,1,1
2,0.00,0.04,0.61,0.02,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0.00,0,0
3,0.00,0.00,1.00,0.28,0,0,0,0,15,0,...,0,0,0,0,0,0,0,1.00,1,1
4,0.00,0.17,0.03,0.02,0,0,0,0,11,0,...,1,0,0,0,0,0,0,0.75,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,0.00,0.06,0.01,0.01,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0.00,0,0
22540,0.00,0.00,0.01,0.01,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0.18,0,0
22541,0.00,0.00,0.00,0.00,0,0,0,2,15,1,...,0,0,0,0,0,0,0,0.20,1,1
22542,0.00,0.01,0.00,0.00,0,0,0,0,21,0,...,0,0,0,0,0,0,0,0.33,0,0


# after trying various techniques i am getting best accuracy of 89% with logistic regression for the testing data set.